<a href="https://colab.research.google.com/github/ishiwish/My-Projects/blob/main/kpopgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests

#Get the pdf
pdf_path= "Kpop_Rag_Model.pdf"

if not os.path.exists(pdf_path):
  print(f"[INFO]File doesn't exist...downloading")

  #enter url for pdf - Using the direct download link from Google Drive
  url = "https://drive.google.com/uc?export=download&id=1HaqzSoCWYEhH2OuEfS66W7MnG1pyKfzB"

  filename = pdf_path

  response = requests.get(url, stream=True)

  if response.status_code == 200:
    with open(filename, 'wb') as f:
      for chunk in response.iter_content(chunk_size=8192):
          f.write(chunk)
    print(f"FILE HAS BEEN DOWNLOADED AND SAVED AS {filename}")
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")
else:
   print(f"File {pdf_path} exists")

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create a text input widget for the query
query_input = widgets.Textarea(
    placeholder='Enter your query here...',
    description='Query:',
    disabled=False,
    layout={'width': '500px', 'height': '100px'}
)

# Create an output widget to display results
output_area = widgets.Output()

# Create a button to trigger the search
search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to perform search',
    icon='search' # (FontAwesome icons available: https://fontawesome.com/icons?d=gallery&c=all)
)

# Function to handle button click
def on_button_clicked(b):
    with output_area:
        output_area.clear_output()
        query = query_input.value
        if query:
            print(f"Searching for: '{query}'\n")
            scores, indices = retrieve_relevant_sources(query=query,
                                                        embeddings=embeddings,
                                                        n_resources_to_return=3) # You can adjust the number of results here

            print("Top relevant chunks:")
            for score, idx in zip(scores, indices):
                print(f"Score: {score:.4f}")
                print("Text:")
                print_wrapped(text_chunk_and_embedding_df.iloc[idx.item()]["sentence_chunk"])
                print(f"Page Number: {text_chunk_and_embedding_df.iloc[idx.item()]['page_number']}")
                print("-" * 50) # Separator for readability

# Link the button click to the function
search_button.on_click(on_button_clicked)

# Display the widgets
display(query_input, search_button, output_area)

In [ ]:
display(pages_and_texts)

In [ ]:
display(pages_and_texts)

In [ ]:
import fitz
from tqdm.auto import tqdm
import os

def text_formatter(text:str) -> str:
  """Performs Minor formatting on text"""

  cleaned_text = text.replace("\n", " ").strip()

  #potentially more text formatting fuctions can go here

  return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
  if not os.path.exists(pdf_path):
    print(f"Error: File not found at {pdf_path}")
    return []

  doc = fitz.open(pdf_path)
  if not doc:
    print(f"Error: Could not open PDF file at {pdf_path}")
    return []

  print(f"Successfully opened PDF with {doc.page_count} pages.")
  pages_and_texts = []
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text = text)
    pages_and_texts.append({"page_number": page_number - 1,
                           "page_char_count": len(text),
                           "page_word_count": len(text.split(" ")),
                           "page_sentence_count_raw": len(text.split(". ")),
                           "page_token_count": len(text)/ 4,
                           "text": text}
                           )
  return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path)

WE GOT A PDF LETS OPEN IT

In [ ]:
  !pip install --upgrade pymupdf

In [ ]:
import fitz
from tqdm.auto import tqdm
import os

def text_formatter(text:str) -> str:
  """Performs Minor formatting on text"""

  cleaned_text = text.replace("\n", " ").strip()

  #potentially more text formatting fuctions can go here

  return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
  if not os.path.exists(pdf_path):
    print(f"Error: File not found at {pdf_path}")
    return []

  doc = fitz.open(pdf_path)
  if not doc:
    print(f"Error: Could not open PDF file at {pdf_path}")
    return []

  print(f"Successfully opened PDF with {doc.page_count} pages.")
  pages_and_texts = []
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text = text)
    pages_and_texts.append({"page_number": page_number - 1,
                           "page_char_count": len(text),
                           "page_word_count": len(text.split(" ")),
                           "page_sentence_count_raw": len(text.split(". ")),
                           "page_token_count": len(text)/ 4,
                           "text": text}
                           )
  return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path)


In [ ]:
import random
random.sample(pages_and_texts, 1)

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

In [ ]:
df.describe().round(2)

In [ ]:
from spacy.lang.en import English

nlp = English()

nlp.add_pipe("sentencizer")

doc = nlp("This is a sentence. This is another sentence. I like Kpop")

assert len(list(doc.sents)) == 3

list(doc.sents)

In [ ]:
pages_and_texts[0]

In [ ]:
import fitz

pdf_path = "Kpop_Corrected_Groups_and_Soloists.pdf" # Make sure this is the correct path to your PDF

try:
    doc = fitz.open(pdf_path)
    if not doc:
        print(f"Error: Could not open PDF file at {pdf_path}")
    else:
        print(f"Successfully opened PDF with {doc.page_count} pages.")
        for page_number in range(doc.page_count):
            page = doc.load_page(page_number)
            text_blocks = page.get_text("blocks") # Get text blocks

            print(f"\n--- Page {page_number + 1} ---")
            if text_blocks:
                # Print the first few text blocks to inspect
                for i, block in enumerate(text_blocks[:5]): # Displaying up to 5 blocks
                    print(f"Block {i}: {block[4]}") # block[4] contains the text
            else:
                print("No text blocks found on this page.")

except FileNotFoundError:
    print(f"Error: File not found at {pdf_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
display(pages_and_texts[0])

In [ ]:
display(pages_and_texts)

In [ ]:
for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)
  item["sentences"] = [str(sent) for sent in item["sentences"]]
  item["page_sentence_count_spacy"] = len(item["sentences"])

In [ ]:
random.sample(pages_and_texts, k=1)

Now that the text is in chunks, we can embed each chunk. Embedding is the process of converting text into numerical representations (vectors) that capture their semantic meaning. These embeddings can then be used for various downstream tasks like semantic search, clustering, or as input for other machine learning models.

We will use a Sentence Transformer model from the `sentence_transformers` library to create the embeddings. If you don't have it installed, the following cell will install it.

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

In [ ]:
num_sentence_chunk_size = 5

def split_list(input_list: list[str],
               slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
  """Splits a list into sublists of a specified size."""

  return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))
test_chunks = split_list(test_list)
print(test_chunks)

In [ ]:
!pip install sentence-transformers

In [ ]:
for item in tqdm(pages_and_texts):
  item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                       slice_size=num_sentence_chunk_size)

item["num_chunks"]= len(item["sentence_chunks"])

In [ ]:
random.sample(pages_and_texts, k=1)

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

In [ ]:
from posixpath import join
import re

pages_and_chunks = []
for item in tqdm(pages_and_texts):
  for sentence_chunk in item["sentence_chunks"]:
   chunk_dict = {}
   chunk_dict["page_number"] = item["page_number"]


   joined_sentence_chunk = "". join(sentence_chunk).replace(" ", " ").strip()
   joined_sentence_chunk = re.sub(r'\. ([A-Z])', r'. \1', joined_sentence_chunk)

   chunk_dict["sentence_chunk"] = joined_sentence_chunk

   chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
   chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
   chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4


   pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

In [ ]:
random.sample(pages_and_chunks, k=1)

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

In [ ]:
import random

min_token_length = 30
# Sample up to 5 rows from the filtered DataFrame
sampled_rows = df[df["chunk_token_count"] <= min_token_length].sample(min(5, len(df[df["chunk_token_count"] <= min_token_length])))

for index, row in sampled_rows.iterrows():
  print(f'Chunk token count:{row["chunk_token_count"]}| Text: {row["sentence_chunk"]}')

In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

In [ ]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device = "cpu")

sentences = ["The sentence transformer library allows for an easy way to create embeddings.",
"Sentences can embedded one by one or in a list.",
"I like Kpop!"]

embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dict.items():
  print(f"Sentence: {sentence}")
  print(f"Embedding: {embedding}")
  print("")

In [ ]:
display(embeddings[0].shape)

In [ ]:
%%time

for item in tqdm(pages_and_chunks_over_min_token_len):
  item["embedding"] = embedding_model.encode(item["sentence_chunk"])

In [ ]:
print(embedding_model.device)

In [ ]:
%%time

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
# text_chunks[419]

In [ ]:
print(len(text_chunks))

In [ ]:
%%time

text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch=32,
                                               convert_to_tensor=True)
text_chunk_embeddings

In [ ]:
pages_and_chunks_over_min_token_len[1]

In [ ]:
text_chunks_and_embeddings = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embedding_df.csv"
text_chunks_and_embeddings.to_csv(embeddings_df_save_path, index=False)

In [ ]:
text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df_load.head()

In [ ]:
import random

import torch
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunk_and_embedding_df = pd.read_csv("text_chunks_and_embedding_df.csv")
# pages_and_chunks = text_chunks_and_embeddings_df_load.to_dict(orient="records")

text_chunk_and_embedding_df['embedding'] = text_chunk_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep= " "))
embeddings = torch.tensor(np.stack(text_chunk_and_embedding_df["embedding"].tolist(), axis=0),dtype=torch.float32).to(device)

text_chunk_and_embedding_df

In [ ]:
embeddings.shape

In [ ]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device = device)

In [ ]:
text_chunk_and_embedding_df["embedding"]

In [ ]:
embeddings

In [ ]:
query = "tvxq"
print(f"Query: {query}")
# Ensure query_embedding is on the same device as embeddings
query_embedding = embedding_model.encode(query, convert_to_tensor=True).to(device)

from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding,b=embeddings)[0]
end_time = timer()
print(f"[INFO] Time taken to get dot scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds")

top_results = torch.topk(dot_scores, k=min(5, len(embeddings)))
display(top_results)

In [ ]:
pages_and_chunks[1]

In [ ]:
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)
print(f"Embeddings shape: {larger_embeddings.shape}")

start_time = timer()
dot_scores = util.dot_score(a=query_embedding,b=larger_embeddings)[0]
end_time = timer()

print(f"[INFO] Time taken to get dot scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds")


In [ ]:
device

In [ ]:
import textwrap

def print_wrapped(text, wrap_length=80):
  wrapped_text = textwrap.fill(text,wrap_length)
  print(wrapped_text)

In [ ]:
query = "enhypen"
print(f"Query: '{query}'\n")

print("Results:")

# Assuming 'top_results' is defined from a previous cell (e.g., using torch.topk)
# and 'text_chunk_and_embedding_df' is your DataFrame with text chunks

for score, idx in zip(top_results[0], top_results[1]):
  print(f"Score: {score:.4f}")
  print("Text: ")
  # Access the sentence chunk from the DataFrame using the index
  print_wrapped(text_chunk_and_embedding_df.iloc[idx.item()]["sentence_chunk"])
  print(f"Page Number: {text_chunk_and_embedding_df.iloc[idx.item()]['page_number']}")
  print("\n")

  print("")

In [ ]:
def retrieve_relevant_sources(query: str,
                              embeddings: torch.tensor,
                              model: SentenceTransformer = embedding_model,
                              n_resources_to_return: int = 5,
                              print_time: bool = True):

  query_embedding = model.encode(query, convert_to_tensor=True)

  start_time = timer()
  dot_scores = util.dot_score(a=query_embedding,b=embeddings)[0]
  end_time = timer()


  if print_time:
    print(f"[INFO] Time taken to get dot scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds")

  scores, indices = torch.topk(dot_scores,
                               k=n_resources_to_return)

  return scores, indices

In [ ]:
# Example usage of the retrieve_relevant_sources function
query = "what are the kpop generations?"
n_results = 3 # Number of relevant chunks to retrieve

scores, indices = retrieve_relevant_sources(query=query,
                                            embeddings=embeddings,
                                            n_resources_to_return=n_results)

print(f"Query: '{query}'\n")
print("Top relevant chunks:")

# Loop through the top results and display the text and page number
for score, idx in zip(scores, indices):
  print(f"Score: {score:.4f}")
  print("Text:")
  print_wrapped(text_chunk_and_embedding_df.iloc[idx.item()]["sentence_chunk"])
  print(f"Page Number: {text_chunk_and_embedding_df.iloc[idx.item()]['page_number']}")
  print("-" * 50) # Separator for readability

Now we can load an embedding model and create embeddings for each chunk.